## 1. Environment Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# --- Configuration ---
REPO_URL = "https://github.com/armanfeili/novartis_datathon_2025.git"
PROJECT_PATH = "/content/drive/MyDrive/novartis_datathon_2025"
BRANCH = "Arman" # Change this if you are working on a different branch
# ---------------------

if not os.path.exists(PROJECT_PATH):
    print(f"Cloning repository to {PROJECT_PATH}...")
    !git clone {REPO_URL} {PROJECT_PATH}
else:
    print(f"Repository exists at {PROJECT_PATH}. Pulling latest changes...")
    %cd {PROJECT_PATH}
    !git pull origin {BRANCH}

%cd {PROJECT_PATH}

In [ ]:
# Install Dependencies
!pip install -r requirements.txt

## 2. Run Experiments

In [ ]:
import sys
import os

# Ensure project root is in path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from src.train import run_experiment

In [ ]:
# --- Experiment Configuration ---
config_path = "configs/model_lgbm.yaml"
run_name = "colab_run_01"
# --------------------------------

print(f"Running experiment: {run_name} with config: {config_path}")
run_experiment(config_path=config_path, run_name=run_name)

## 3. Checkpoints & Exports
Checkpoints are saved in the `models/` directory and logs in `logs/`.

In [ ]:
!ls -R models/
!ls -R logs/